# Export Results for TP Report (Simple Version)
This notebook exports all metrics and creates visualizations for your report.

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure MLflow
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup complete")

## 1. Get Model Results

In [ ]:
# Get all runs from sentiment_analysis experiment
experiment = client.get_experiment_by_name("/Users/nmoccagatta@itba.edu.ar/sentiment_analysis")

if experiment:
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.f1_score DESC"]
    )
    
    # Create comparison table
    results = []
    for run in runs:
        results.append({
            "Model": run.info.run_name,
            "F1 Score": round(run.data.metrics.get("f1_score", 0), 4),
            "Precision": round(run.data.metrics.get("precision", 0), 4),
            "Recall": round(run.data.metrics.get("recall", 0), 4)
        })
    
    df_results = pd.DataFrame(results)
    
    print(f"✅ Found {len(df_results)} model runs\n")
    print(f"🏆 Best Model: {df_results.iloc[0]['Model']}")
    print(f"📊 Best F1 Score: {df_results.iloc[0]['F1 Score']}\n")
    
    display(df_results)
else:
    print("❌ Experiment not found")
    df_results = None

## 2. Visualization 1: F1 Score Comparison

In [ ]:
if df_results is not None:
    plt.figure(figsize=(14, 6))
    colors = ['#2ecc71' if i == 0 else '#3498db' for i in range(len(df_results))]
    bars = plt.bar(df_results['Model'], df_results['F1 Score'], color=colors)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.4f}',
                 ha='center', va='bottom', fontsize=9)
    
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('F1 Score (Macro)', fontsize=12)
    plt.xlabel('Model', fontsize=12)
    plt.title('Model Comparison - F1 Score Performance', fontsize=14, fontweight='bold')
    plt.ylim(0, max(df_results['F1 Score']) * 1.1)
    plt.tight_layout()
    plt.show()
    
    print("💡 Right-click on the chart above and 'Save Image As' to download")

## 3. Visualization 2: All Metrics Comparison

In [ ]:
if df_results is not None:
    metrics_df = df_results[['Model', 'F1 Score', 'Precision', 'Recall']].set_index('Model')
    
    fig, ax = plt.subplots(figsize=(14, 6))
    metrics_df.plot(kind='bar', ax=ax, color=['#e74c3c', '#3498db', '#2ecc71'])
    
    plt.ylabel('Score', fontsize=12)
    plt.xlabel('Model', fontsize=12)
    plt.title('Model Performance - All Metrics Comparison', fontsize=14, fontweight='bold')
    plt.legend(title='Metrics', fontsize=10)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1.0)
    plt.tight_layout()
    plt.show()
    
    print("💡 Right-click on the chart above and 'Save Image As' to download")

## 4. Visualization 3: Model Type Comparison

In [ ]:
if df_results is not None:
    # Categorize models by type
    def categorize_model(model_name):
        if 'LR' in model_name:
            return 'Logistic Regression'
        elif 'SVM' in model_name:
            return 'Linear SVM'
        elif 'NB' in model_name:
            return 'Naive Bayes'
        else:
            return 'Other'
    
    df_results['Model Type'] = df_results['Model'].apply(categorize_model)
    
    # Calculate average by model type
    model_type_avg = df_results.groupby('Model Type')[['F1 Score', 'Precision', 'Recall']].mean()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    model_type_avg.plot(kind='bar', ax=ax, color=['#e74c3c', '#3498db', '#2ecc71'])
    
    plt.ylabel('Average Score', fontsize=12)
    plt.xlabel('Model Type', fontsize=12)
    plt.title('Average Performance by Model Type', fontsize=14, fontweight='bold')
    plt.legend(title='Metrics', fontsize=10)
    plt.xticks(rotation=0)
    plt.ylim(0, 1.0)
    plt.tight_layout()
    plt.show()
    
    print("💡 Right-click on the chart above and 'Save Image As' to download")

## 5. Summary Statistics

In [ ]:
if df_results is not None:
    summary = {
        "Total Models Trained": len(df_results),
        "Best Model": df_results.iloc[0]['Model'],
        "Best F1 Score": df_results.iloc[0]['F1 Score'],
        "Best Precision": df_results.iloc[0]['Precision'],
        "Best Recall": df_results.iloc[0]['Recall'],
        "Average F1 Score": round(df_results['F1 Score'].mean(), 4),
        "F1 Score Std Dev": round(df_results['F1 Score'].std(), 4),
        "Best Model Type": df_results.iloc[0]['Model Type']
    }
    
    print("\n" + "="*60)
    print("SUMMARY STATISTICS FOR YOUR TP REPORT")
    print("="*60 + "\n")
    
    for key, value in summary.items():
        print(f"{key:25s}: {value}")
    
    print("\n" + "="*60)

## 6. Report Text Template

In [ ]:
if df_results is not None:
    report_text = f"""
RESULTADOS EXPERIMENTALES
========================

Se entrenaron {summary['Total Models Trained']} variantes de modelos de clasificación para análisis de sentimientos:
- 3 variantes de Regresión Logística (C=1, 5, 10)
- 4 variantes de SVM Lineal (C=0.1, 1, 10, 50)  
- 4 variantes de Naive Bayes Multinomial (alpha=0.2, 0.5, 1, 2)

MEJOR MODELO
------------
Modelo: {summary['Best Model']}
Tipo: {summary['Best Model Type']}
F1 Score (macro): {summary['Best F1 Score']}
Precision (macro): {summary['Best Precision']}
Recall (macro): {summary['Best Recall']}

ANÁLISIS COMPARATIVO
-------------------
F1 Score promedio: {summary['Average F1 Score']}
Desviación estándar: {summary['F1 Score Std Dev']}

El modelo {summary['Best Model']} mostró el mejor desempeño con un F1 Score de {summary['Best F1 Score']},
superando al promedio general por {round(summary['Best F1 Score'] - summary['Average F1 Score'], 4)} puntos.

MLOPS IMPLEMENTADO
------------------
✅ Tracking de experimentos con MLflow
✅ Arquitectura Medallion (Bronze → Silver → Gold)
✅ Registro de modelos en Unity Catalog (workspace.sentiment_analysis.sentiment_classifier)
✅ Búsqueda sistemática de hiperparámetros (12 variantes)
✅ Validación estratificada 80/20
✅ Vectorización TF-IDF con 50,000 features y n-grams (1,2)
✅ Métricas de calidad de datos en cada etapa del pipeline
"""
    
    print(report_text)

## 7. Export Data Table (Copy-Paste Ready)

In [ ]:
if df_results is not None:
    print("\n📋 COPY THIS TABLE FOR YOUR REPORT:\n")
    print(df_results.to_markdown(index=False))
    print("\n💡 This is in Markdown format - you can paste it directly into your report!")

## ✅ Instructions

### To Save Charts:
1. Right-click on each chart above
2. Select "Save Image As..."
3. Save as PNG files

### To Save Data:
1. Copy the markdown table from cell 7
2. Copy the summary statistics from cell 5
3. Copy the report text from cell 6

### What to Include in Your Report:
1. **Architecture Diagram** (from MLflow_Pipeline_Explanation.md)
2. **Model Comparison Table** (from cell 7)
3. **3 Charts** (saved from cells 2, 3, 4)
4. **Summary Statistics** (from cell 5)
5. **Screenshots** of MLflow Experiments and Registered Model
6. **Your Analysis** using the template from cell 6